In [4]:
import torch
# import torchvision

import pandas as pd
import numpy as np

In [35]:

cifar10n_labels = torch.load("CIFAR-100_human.pt")

print(cifar10n_labels.keys())
print(cifar10n_labels["noisy_label"][:10])

classname_dict = {
    0: 'apple', 1: 'aquarium_fish', 2: 'baby', 3: 'bear', 4: 'beaver', 5: 'bed', 6: 'bee', 7: 'beetle', 8: 'bicycle', 9: 'bottle', 10: 'bowl', 11: 'boy', 12: 'bridge', 13: 'bus', 14: 'butterfly', 15: 'camel', 16: 'can', 17: 'castle', 18: 'caterpillar', 19: 'cattle', 20: 'chair', 21: 'chimpanzee', 22: 'clock', 23: 'cloud', 24: 'cockroach', 25: 'couch', 26: 'crab', 27: 'crocodile', 28: 'cup', 29: 'dinosaur', 30: 'dolphin', 31: 'elephant', 32: 'flatfish', 33: 'forest', 34: 'fox', 35: 'girl', 36: 'hamster', 37: 'house', 38: 'kangaroo', 39: 'keyboard', 40: 'lamp', 41: 'lawn_mower', 42: 'leopard', 43: 'lion', 44: 'lizard', 45: 'lobster', 46: 'man', 47: 'maple_tree', 48: 'motorcycle', 49: 'mountain', 50: 'mouse', 51: 'mushroom', 52: 'oak_tree', 53: 'orange', 54: 'orchid', 55: 'otter', 56: 'palm_tree', 57: 'pear', 58: 'pickup_truck', 59: 'pine_tree', 60: 'plain', 61: 'plate', 62: 'poppy', 63: 'porcupine', 64: 'possum', 65: 'rabbit', 66: 'raccoon', 67: 'ray', 68: 'road', 69: 'rocket', 70: 'rose', 71: 'sea', 72: 'seal', 73: 'shark', 74: 'shrew', 75: 'skunk', 76: 'skyscraper', 77: 'snail', 78: 'snake', 79: 'spider', 80: 'squirrel', 81: 'streetcar', 82: 'sunflower', 83: 'sweet_pepper', 84: 'table', 85: 'tank', 86: 'telephone', 87: 'television', 88: 'tiger', 89: 'tractor', 90: 'train', 91: 'trout', 92: 'tulip', 93: 'turtle', 94: 'wardrobe', 95: 'whale', 96: 'willow_tree', 97: 'wolf', 98: 'woman', 99: 'worm'}

superclass_dict = {
    0: 'aquatic animals',
    1: 'fish',
    2: 'flowers',
    3: 'food containers',
    4: 'fruit and vegetables',
    5: 'household electrical devices',
    6: 'household furniture',
    7: 'insects',
    8: 'large carnivores',
    9: 'large man-made outdoor things',
    10: 'large natural outdoor scenes',
    11: 'large omnivores and herbivores',
    12: 'medium-sized mammals',
    13: 'non-insect invertebrates',
    14: 'people',
    15: 'reptiles',
    16: 'small mammals',
    17: 'trees',
    18: 'vehicles 1',
    19: 'vehicles 2'
}

def sparse2coarse(targets):
    """Convert Pytorch CIFAR100 sparse targets to coarse targets.

    Usage:
        trainset = torchvision.datasets.CIFAR100(path)
        trainset.targets = sparse2coarse(trainset.targets)
    """
    coarse_labels = np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,  
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,  
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5, 
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10, 
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17, 
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,  
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13, 
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19, 
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])
    return coarse_labels[targets]
# reverse the dictionary
#classname_dict = {v: k for k, v in classname_dict.items()}


dict_keys(['clean_label', 'noisy_label', 'clean_coarse_label', 'noisy_coarse_label'])
[19 63 10 11  1 86 90 28 23 31]


In [36]:
for split in ["val","test"]:
    train_df = pd.read_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100.txt", sep=" ", header=None)
    train_df.columns = ["filename", "label_orig"]
    train_df.set_index("filename", inplace=True)
    train_df["label_coarse"] = sparse2coarse(train_df["label_orig"])
    print(train_df.head())
    train_df.to_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100n_cleancoarse.txt", sep=" ", header=False, columns=["label_coarse"])

                              label_orig  label_coarse
filename                                              
cifar100/test/apple/0040.png           0             4
cifar100/test/apple/0074.png           0             4
cifar100/test/apple/0089.png           0             4
cifar100/test/apple/0080.png           0             4
cifar100/test/apple/0049.png           0             4
                              label_orig  label_coarse
filename                                              
cifar100/test/apple/0076.png           0             4
cifar100/test/apple/0009.png           0             4
cifar100/test/apple/0063.png           0             4
cifar100/test/apple/0008.png           0             4
cifar100/test/apple/0097.png           0             4


In [37]:
for split in ["train"]:
    dataset_list = []
    count_dict = {}

    # combine train and test labels
    #labels = trainset.targets

    for i,_ in enumerate(cifar10n_labels["clean_label"]):
        label = cifar10n_labels["clean_label"][i]
        classname = classname_dict[label]
        count_dict[classname] = count_dict.get(classname, 0) + 1

        # if count_dict[classname] == 1:
        #     # save the image
        #     img = torchvision.transforms.ToTensor()(trainset.data[i])
        #     torchvision.utils.save_image(img, f"{classname}-{count_dict[classname]:04}.png")
        
        if split in ["val","test"]: source_folder = "test"
        else: source_folder = "train"
        filename = f"cifar100/{source_folder}/{classname}/{count_dict[classname]:>04}.png"
        #print(cifar10n_labels)
        dataset_list.append(
            {
                "filename": filename,
                #"label_orig": label,
                "label_clean_fine": cifar10n_labels["clean_label"][i],
                "label_noisy_fine": cifar10n_labels["noisy_label"][i]
            }
        )

    print(count_dict)

    # create dataframe
    df = pd.DataFrame(dataset_list)

    print(f"Found {len(df)} images in total")

    df.set_index("filename", inplace=True)

    # load txt file data/benchmark_imglist/cifar10/train_cifar10.txt
    train_df = pd.read_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100.txt", sep=" ", header=None)

    # rename columns to match
    train_df.columns = ["filename", "label_orig"]

    # update index of both dataframes
    train_df.set_index("filename", inplace=True)
    
    print(f"Found {len(train_df)} images in {split} set")

    #sort both dataframes by index
    #train_df.sort_index(inplace=True)
    #df.sort_index(inplace=True)
    print(train_df.head())
    print(df.head())

    # create new dataframe with updated labels but the order of the train_df
    #train_df_noisy = pd.concat([train_df, df], axis=1, join="inner")
    train_df_noisy = train_df.merge(df, how="left", left_index=True, right_index=True)
    print(train_df_noisy.head())

    # remove column 1
    train_df_noisy.drop(columns=["label_clean_fine"], inplace=True)

    #print(train_df.head())
    #print(train_df_noisy.head())
    # remove rows from df_new that are not in df
    train_df_noisy = train_df_noisy[train_df_noisy.index.isin(train_df.index)]

    print(f"After merging, the number of rows in the train_df_noisy is {len(train_df_noisy)}")

    print(train_df_noisy.head())

    # save the new dataframes as txt files, selecting the label_worse column
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100n_noisyfine.txt", sep=" ", header=False, columns=["label_noisy_fine"])

{'cattle': 500, 'dinosaur': 500, 'apple': 500, 'boy': 500, 'aquarium_fish': 500, 'telephone': 500, 'train': 500, 'cup': 500, 'cloud': 500, 'elephant': 500, 'keyboard': 500, 'willow_tree': 500, 'sunflower': 500, 'castle': 500, 'sea': 500, 'bicycle': 500, 'wolf': 500, 'squirrel': 500, 'shrew': 500, 'pine_tree': 500, 'rose': 500, 'television': 500, 'table': 500, 'possum': 500, 'oak_tree': 500, 'leopard': 500, 'maple_tree': 500, 'rabbit': 500, 'chimpanzee': 500, 'clock': 500, 'streetcar': 500, 'cockroach': 500, 'snake': 500, 'lobster': 500, 'mountain': 500, 'palm_tree': 500, 'skyscraper': 500, 'tractor': 500, 'shark': 500, 'butterfly': 500, 'bottle': 500, 'bee': 500, 'chair': 500, 'woman': 500, 'hamster': 500, 'otter': 500, 'seal': 500, 'lion': 500, 'mushroom': 500, 'girl': 500, 'sweet_pepper': 500, 'forest': 500, 'crocodile': 500, 'orange': 500, 'tulip': 500, 'mouse': 500, 'camel': 500, 'caterpillar': 500, 'man': 500, 'skunk': 500, 'kangaroo': 500, 'raccoon': 500, 'snail': 500, 'rocket': 

In [38]:
for split in ["train"]:
    dataset_list = []
    count_dict = {}

    # combine train and test labels
    #labels = trainset.targets

    for i,_ in enumerate(cifar10n_labels["clean_label"]):
        label = cifar10n_labels["clean_label"][i]
        classname = classname_dict[label]
        count_dict[classname] = count_dict.get(classname, 0) + 1

        # if count_dict[classname] == 1:
        #     # save the image
        #     img = torchvision.transforms.ToTensor()(trainset.data[i])
        #     torchvision.utils.save_image(img, f"{classname}-{count_dict[classname]:04}.png")
        
        if split in ["val","test"]: source_folder = "test"
        else: source_folder = "train"
        filename = f"cifar100/{source_folder}/{classname}/{count_dict[classname]:>04}.png"
        #print(cifar10n_labels)
        dataset_list.append(
            {
                "filename": filename,
                #"label_orig": label,
                "label_clean_fine": cifar10n_labels["clean_label"][i],
                "label_noisy_fine": cifar10n_labels["noisy_label"][i],
                "label_clean_coarse": cifar10n_labels["clean_coarse_label"][i],
                "label_noisy_coarse": cifar10n_labels["noisy_coarse_label"][i],
            }
        )

    print(count_dict)

    # create dataframe
    df = pd.DataFrame(dataset_list)

    print(f"Found {len(df)} images in total")

    df.set_index("filename", inplace=True)

    # load txt file data/benchmark_imglist/cifar10/train_cifar10.txt
    train_df = pd.read_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100.txt", sep=" ", header=None)

    # rename columns to match
    train_df.columns = ["filename", "label_orig"]

    # update index of both dataframes
    train_df.set_index("filename", inplace=True)
    
    print(f"Found {len(train_df)} images in {split} set")

    #sort both dataframes by index
    #train_df.sort_index(inplace=True)
    #df.sort_index(inplace=True)
    print(train_df.head())
    print(df.head())

    # create new dataframe with updated labels but the order of the train_df
    #train_df_noisy = pd.concat([train_df, df], axis=1, join="inner")
    train_df_noisy = train_df.merge(df, how="left", left_index=True, right_index=True)
    print(train_df_noisy.head())

    # remove column 1
    train_df_noisy.drop(columns=["label_clean_fine"], inplace=True)

    #print(train_df.head())
    #print(train_df_noisy.head())
    # remove rows from df_new that are not in df
    train_df_noisy = train_df_noisy[train_df_noisy.index.isin(train_df.index)]

    print(f"After merging, the number of rows in the train_df_noisy is {len(train_df_noisy)}")

    print(train_df_noisy.head())

    # save the new dataframes as txt files, selecting the label_worse column
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100n_noisycoarse.txt", sep=" ", header=False, columns=["label_noisy_coarse"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100n_noisyfine.txt", sep=" ", header=False, columns=["label_noisy_fine"])
    train_df_noisy.to_csv(f"../../benchmark_imglist/cifar100/{split}_cifar100n_cleancoarse.txt", sep=" ", header=False, columns=["label_clean_coarse"])

{'cattle': 500, 'dinosaur': 500, 'apple': 500, 'boy': 500, 'aquarium_fish': 500, 'telephone': 500, 'train': 500, 'cup': 500, 'cloud': 500, 'elephant': 500, 'keyboard': 500, 'willow_tree': 500, 'sunflower': 500, 'castle': 500, 'sea': 500, 'bicycle': 500, 'wolf': 500, 'squirrel': 500, 'shrew': 500, 'pine_tree': 500, 'rose': 500, 'television': 500, 'table': 500, 'possum': 500, 'oak_tree': 500, 'leopard': 500, 'maple_tree': 500, 'rabbit': 500, 'chimpanzee': 500, 'clock': 500, 'streetcar': 500, 'cockroach': 500, 'snake': 500, 'lobster': 500, 'mountain': 500, 'palm_tree': 500, 'skyscraper': 500, 'tractor': 500, 'shark': 500, 'butterfly': 500, 'bottle': 500, 'bee': 500, 'chair': 500, 'woman': 500, 'hamster': 500, 'otter': 500, 'seal': 500, 'lion': 500, 'mushroom': 500, 'girl': 500, 'sweet_pepper': 500, 'forest': 500, 'crocodile': 500, 'orange': 500, 'tulip': 500, 'mouse': 500, 'camel': 500, 'caterpillar': 500, 'man': 500, 'skunk': 500, 'kangaroo': 500, 'raccoon': 500, 'snail': 500, 'rocket': 